In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%%capture
!pip install transformers[torch]
!pip install datasets
!pip install pyarrow

In [5]:
import pandas as pd

In [6]:
my_data = pd.read_csv("/content/drive/MyDrive/Data Science and Machine Learning/Project_Data_Science/training_data.csv")

In [7]:
my_data.head()

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1


In [8]:
my_data.shape

(4800, 3)

In [13]:
from transformers import AutoModel, AutoTokenizer
from typing import Dict

In [10]:
%%capture
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-french-europeana-cased")

In [14]:
def process_data(row) -> Dict:
    # Clean the text
    text = row['sentence']
    text = str(text)
    text = ' '.join(text.split())

    # Get tokens
    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=512)

    # Convert difficulty labels to integers
    difficulty_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
    label = difficulty_mapping.get(row['difficulty'], 0)

    encodings['label'] = label
    encodings['text'] = text

    return encodings

In [15]:
#test
print(process_data({'sentence': "J'aime les films d'actions",'difficulty': 'B2'}))

{'input_ids': [2, 46, 11, 4694, 361, 16909, 71, 11, 3294, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [25]:
# Store the encodings into an array to generate dataset
processed_data = []

for i in range(len(my_data[:4800])):
        processed_data.append(process_data(my_data.iloc[i]))

In [24]:
from sklearn.model_selection import train_test_split

In [22]:
new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(new_df,test_size=0.2,random_state=2022)

In [23]:
import pyarrow as pa
from datasets import Dataset

In [20]:
train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [29]:
print(type(train_hg))

<class 'datasets.arrow_dataset.Dataset'>


In [21]:
from transformers import AutoModelForSequenceClassification

In [26]:
%%capture
model = AutoModel.from_pretrained("dbmdz/bert-base-french-europeana-cased")

In [27]:
model2 = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-french-europeana-cased",num_labels=6)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-french-europeana-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
from transformers import TrainingArguments, Trainer

In [31]:
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/Data Science and Machine Learning/Project_Data_Science/result", evaluation_strategy="epoch")

trainer = Trainer(model=model2,args=training_args,train_dataset=train_hg,eval_dataset=valid_hg,tokenizer=tokenizer)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.110786
2,1.252400,1.040913
3,0.815100,1.226013


TrainOutput(global_step=1440, training_loss=0.8599172486199274, metrics={'train_runtime': 907.687, 'train_samples_per_second': 12.692, 'train_steps_per_second': 1.586, 'total_flos': 2368084543488000.0, 'train_loss': 0.8599172486199274, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.2260127067565918,
 'eval_runtime': 24.4801,
 'eval_samples_per_second': 39.216,
 'eval_steps_per_second': 4.902,
 'epoch': 3.0}

In [ ]:
model.save_pretrained('/content/drive/MyDrive/Data Science and Machine Learning/Project_Data_Science/train_model_bert/')